In [29]:
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('datos/caba_para_mapa.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd)
dataframe_rdd = dataframe.rdd

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

# USE CABA PORQUE LSH TIRA ERROR CUANDO ENCUENTRA NANS

In [4]:
dataframe.dtypes

[('created_on', 'timestamp'),
 ('property_type', 'string'),
 ('place_name', 'string'),
 ('state_name', 'string'),
 ('lat-lon', 'string'),
 ('lat', 'double'),
 ('lon', 'double'),
 ('price', 'int'),
 ('currency', 'string'),
 ('price_aprox_local_currency', 'double'),
 ('price_aprox_usd', 'int'),
 ('surface_total_in_m2', 'int'),
 ('surface_covered_in_m2', 'int'),
 ('price_usd_per_m2', 'int'),
 ('price_per_m2', 'double'),
 ('floor', 'int'),
 ('rooms', 'int'),
 ('expenses', 'int'),
 ('properati_url', 'string'),
 ('description', 'string'),
 ('title', 'string'),
 ('dist_a_subte', 'double'),
 ('dist_a_tren', 'double'),
 ('dist_a_univ', 'double'),
 ('dist_a_villa', 'double'),
 ('dist_a_zona_anegada', 'double')]

In [5]:
from pyspark.ml.feature import VectorAssembler
#preparo las columnas para pasarle lsh
assembler = VectorAssembler(inputCols=["surface_total_in_m2","dist_a_subte"], outputCol="vector2")
dataframe = assembler.transform(dataframe)

In [6]:
dataframe.select("vector2").show()

+-----------+
|    vector2|
+-----------+
| [29.0,2.7]|
| [47.0,7.3]|
| [47.0,7.3]|
| [86.0,5.8]|
| [65.0,9.3]|
| [35.0,1.9]|
|[153.0,4.5]|
| [40.0,3.7]|
| [40.0,3.7]|
| [40.0,3.7]|
|[470.0,8.7]|
| [70.0,7.0]|
| [47.0,4.3]|
| [48.0,4.3]|
| [33.0,4.3]|
| [28.0,4.3]|
| [26.0,4.3]|
| [48.0,4.3]|
| [50.0,4.3]|
|[270.0,9.4]|
+-----------+
only showing top 20 rows



In [7]:
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

key = Vectors.dense([51.3,0.62])

brp = BucketedRandomProjectionLSH(inputCol="vector2", outputCol="bucket", bucketLength=500.0,
                                  numHashTables=2)#numHashTables: cantidad de ORs, todavia no esta implementado las ANDs
model = brp.fit(dataframe)

# Compute the locality sensitive hashes for the input rows, then perform approximate nearest
# neighbor search.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxNearestNeighbors(transformedA, key, 2)`
print("Approximately searching dfA for 20 nearest neighbors of",key)
resultado=model.approxNearestNeighbors(dataframe, key, 20,'distancia').select('place_name','vector2','bucket','distancia','price')
resultado.show()
resultado.describe().show()

('Approximately searching dfA for 20 nearest neighbors of', DenseVector([51.3, 0.62]))
+------------+----------+--------------+-------------------+-------+
|  place_name|   vector2|        bucket|          distancia|  price|
+------------+----------+--------------+-------------------+-------+
|Barrio Norte|[51.0,0.7]|[[0.0], [0.0]]| 0.3104834939251977| 170000|
|   Balvanera|[51.0,0.7]|[[0.0], [0.0]]| 0.3104834939251977| 130000|
|    Recoleta|[51.0,0.7]|[[0.0], [0.0]]| 0.3104834939251977| 140000|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 204585|
|Barrio Norte|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 152000|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 178187|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 181486|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 184786|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 188746|
|    Recoleta|[51.0,0.5]|[[0.0], [0.0]]| 0.3231098884280676| 192705|
|    Recoleta|[5

## Ejemplo de como pasar un feature continuo a discreto

In [151]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
sup= dataframe.select('surface_total_in_m2')
discretizer = QuantileDiscretizer(numBuckets=100, inputCol="surface_total_in_m2", outputCol="result")
result = discretizer.fit(sup).transform(sup)
result.show()

+-------------------+------+
|surface_total_in_m2|result|
+-------------------+------+
|                 29|   5.0|
|                 47|  22.0|
|                 47|  22.0|
|                 86|  52.0|
|                 65|  38.0|
|                 35|  10.0|
|                153|  70.0|
|                 40|  15.0|
|                 40|  15.0|
|                 40|  15.0|
|                470|  84.0|
|                 70|  42.0|
|                 47|  22.0|
|                 48|  23.0|
|                 33|   8.0|
|                 28|   5.0|
|                 26|   4.0|
|                 48|  23.0|
|                 50|  25.0|
|                270|  80.0|
+-------------------+------+
only showing top 20 rows



### Los algoritmos de clasificacion piden labeled points como entrada

In [169]:
bucket= result.select('result')
bucket=bucket.
print bucket
superficie= result[0].surface_total_in_m2
print superficie
pos = LabeledPoint(label,[superficie])
print pos

Column<result['result']>
Column<surface_total_in_m2['surface_total_in_m2']>


ValueError: setting an array element with a sequence.

### Intento con mas de una columna

In [107]:
sup= dataframe.select('surface_total_in_m2','price')
discretizer = QuantileDiscretizer(numBuckets=100, inputCol="surface_total_in_m2", outputCol="resultado")
discretizado = discretizer.fit(sup).transform(sup)

In [108]:
discretizado.select('surface_total_in_m2')
pos = LabeledPoint(label, [1.0, 0.0, 3.0])
print pos

DataFrame[surface_total_in_m2: int, price: int, resultado: double]
(22.0,[1.0,0.0,3.0])


# GradientBoostedTrees no es multilabel <span style="color:red"> Solo predice 0 ó 1 </span>

In [80]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file.
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = dataframe_rdd.randomSplit([0.7, 0.3])
trainingData.take(2)

model = GradientBoostedTrees.trainRegressor(trainingData,
                                            categoricalFeaturesInfo={}, numIterations=3)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(testData.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression GBT model:')
print(model.toDebugString())


[Row(created_on=datetime.datetime(2015, 7, 1, 0, 0), property_type=u'apartment', place_name=u'Palermo', state_name=u'Capital Federal', lat-lon=u'(-34.574594426799997, -58.419254851700003)', lat=-34.5745944268, lon=-58.4192548517, price=71000, currency=u'USD', price_aprox_local_currency=1252759.5, price_aprox_usd=71000, surface_total_in_m2=29, surface_covered_in_m2=29, price_usd_per_m2=2448, price_per_m2=43198.0, floor=None, rooms=1, expenses=1500, properati_url=u'http://www.properati.com.ar/rlpn_venta_departamentos_palermo', description=u'VENTA DEPARTAMENTO DE 1 AMBIENTE CON PATIO EN PALERMO NUEVOentre Kennedy y Oro.Excelente ubicaci\xc3\xb3n frente a la Embajada de Estados Unidos, a metros de los Bosques de Palermo y una cuadra de Av. del LibertadorMuy buen estado general. Pisos de parquet. Amplio ambiente con cocina integrada y ba\xc3\xb1o completo. Peque\xc3\xb1o patio.IMPORTANTE: ESTOS PRECIOS ESTAN SUJETOS A APROBACION DEL PROPIETARIO', title=u'Departamento en Palermo', dist_a_sub

In [121]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

numero= 10
# Load and parse the data file.
data = MLUtils.loadLibSVMFile(sc, "datos/sample_libsvm_data.txt")

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GradientBoostedTrees model.
#  Notes: (a) Empty categoricalFeaturesInfo indicates all features are continuous.
#         (b) Use more iterations in practice.
model = GradientBoostedTrees.trainRegressor(trainingData,
                                            categoricalFeaturesInfo={}, numIterations=5)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
print predictions.take(numero)
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
print ("--------------------------")
print labelsAndPredictions.take(numero)
print ("--------------------------")
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /\
    float(testData.count())
print('Test Mean Squared Error = ' + str(testMSE))
print('Learned regression GBT model:')
print(model.toDebugString())

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
--------------------------
[(1.0, 1.0), (0.0, 0.0), (1.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0)]
--------------------------
Test Mean Squared Error = 0.0416666666667
Learned regression GBT model:
TreeEnsembleModel regressor with 5 trees

  Tree 0:
    If (feature 434 <= 0.0)
     Predict: 0.0
    Else (feature 434 > 0.0)
     Predict: 1.0
  Tree 1:
    Predict: 0.0
  Tree 2:
    Predict: 0.0
  Tree 3:
    Predict: 0.0
  Tree 4:
    Predict: 0.0



#### No le di mas bola a GBT cuando me entere que no era multilabel

# Random Forest

In [147]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = dataframe_rdd.randomSplit([0.7, 0.3])
print trainingData.take(2)

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
print ("--------------------------")
print predictions.take(numero)
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
print ("--------------------------")
print labelsAndPredictions.take(numero)
print ("--------------------------")
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

[Row(created_on=datetime.datetime(2015, 7, 1, 0, 0), property_type=u'apartment', place_name=u'Balvanera', state_name=u'Capital Federal', lat-lon=u'(-34.605761600000001, -58.393765199999997)', lat=-34.6057616, lon=-58.3937652, price=75000, currency=u'USD', price_aprox_local_currency=1323337.5, price_aprox_usd=75000, surface_total_in_m2=49, surface_covered_in_m2=None, price_usd_per_m2=1530, price_per_m2=27006.0, floor=None, rooms=2, expenses=None, properati_url=u'http://www.properati.com.ar/rm0h_venta_departamentos_balvanera_riobamba_300', description=u'-------NO ES DUE\xc3\x91O DIRECTO---------------------------------------------------------Se vende depto de 2 amb de 49 m2 totales, al frente con balc\xc3\xb3n aterrazado. Cocina separada. Muy luminoso y en muy buen estado. Riobamba al 300 entre Corrientes y Sarmiento. A media cuadra de Av. Corrientes y Est. Callao -Subte l\xc3\xadnea B. Y una cuadra de Av. Callao. Riobamba Al 300 Esquina Corrientes, Balvanera.', title=u'Departamentos ven

AssertionError: the data should be RDD of LabeledPoint